# Modeling revisited

## Importing libraries

In [1]:
import pandas as pd
import numpy as np
import scipy
from math import sqrt
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import os
import time

Estimators: regressors

In [98]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.decomposition import PCA


Metrics:

In [224]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import max_error

Cross-validation:

In [99]:
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score, cross_validate
from sklearn.pipeline import Pipeline

In [127]:
pd.set_option("display.min_rows", 100)

In [128]:
pd.set_option("display.max_rows", 400)

In [125]:
pd.set_option('display.max_colwidth', None)

## Reading dataset

In [5]:
datafilenamepath = "C:\\Users\\edidd\\Documents\\Ubiqum\\Data Analytics Course\\Ernst\\data"

In [6]:
data = pd.read_csv(os.path.join(datafilenamepath, "arranque_195_sin_t_obj.csv"))

## Functions

In [8]:
def extract_features(df, p):
    # features = df.drop(list(df.filter(regex='DT\d\d')), axis =1)
    features = df[list(df.columns.drop(list(df.filter(regex='P\d\d|DT\d\d')))) + [p]].dropna()
    return features

In [9]:
def extract_dependent(df, dt, features):
    dependent = df[dt].loc[features.index]
    return dependent

## Hyperparameter selection and model assessment

In [10]:
features_dt30 = extract_features(data, "P30")

In [11]:
dt30 = extract_dependent(data, "DT30", features_dt30)

### Droping outliers

In [32]:
index_drop = dt30.loc[dt30 > 0].index

In [33]:
dt30.drop(index_drop, inplace= True)

In [34]:
features_dt30.drop(index_drop, inplace= True)

### Nested CV

In [85]:
dt30

3     -2.53
4     -2.04
5     -3.57
6     -1.40
7     -2.23
       ... 
294   -1.07
295   -1.33
296   -1.87
297   -0.33
300   -1.80
Name: DT30, Length: 261, dtype: float64

In [86]:
features_dt30

,Potencia Activa U3,Potencia activa U1,Potencia activa U2,Presión alta 1 U1,Presión alta 1 U2,Presión alta 1 U3,Presión alta 2 U1,Presión alta 2 U2,Presión alta 2 U3,Presión baja 1 U1,...,Presión baja 2 U2,Presión baja 2 U3,Tª exterior,Tª impulsion U3,Tª impulsión U1,Tª impulsión U2,Tª retorno U1,Tª retorno U2,Tª retorno U3,P30
3,92.30,89.50,0.40,27.2,14.7,28.3,27.7,14.7,28.4,8.5,...,13.4,8.9,25.071233,13.0,13.0,17.8,18.0,17.8,17.7,144.558333
4,23.05,89.30,0.40,26.9,14.7,3.4,27.5,14.6,32.8,8.3,...,13.3,9.4,26.054267,15.6,11.8,16.3,16.6,16.4,16.3,111.966667
5,30.15,89.60,0.40,27.1,14.9,3.5,27.7,14.6,19.9,8.3,...,14.2,10.1,26.429167,15.7,12.5,17.1,17.4,17.2,17.0,116.683333
6,48.50,65.65,0.40,22.4,15.1,3.3,25.3,14.9,23.8,8.5,...,14.1,8.2,25.197133,11.5,10.2,13.7,13.9,13.8,13.6,90.766667
7,48.80,87.45,0.40,26.4,14.8,-88.8,27.0,14.5,36.1,8.2,...,13.8,8.2,26.425067,15.4,11.5,16.0,16.4,16.0,16.0,105.008333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,0.45,44.00,65.00,19.9,-8.3,-1.0,20.1,20.9,13.6,8.0,...,8.4,13.0,22.977533,14.4,11.0,8.8,14.6,14.5,14.7,100.400000
295,0.45,62.45,65.35,20.8,-8.4,-1.0,23.4,20.8,13.2,7.8,...,8.5,12.9,21.127967,15.6,11.1,10.0,15.8,15.7,15.6,118.333333
296,0.45,62.45,84.15,21.1,24.1,-1.0,23.5,24.1,12.8,8.1,...,7.7,12.9,20.635067,16.1,11.6,9.2,16.3,16.2,16.1,133.800415
297,70.85,63.40,0.45,21.1,14.5,31.9,23.5,14.4,34.5,7.7,...,12.7,8.7,24.691800,12.2,10.7,15.0,15.3,15.0,14.8,120.867331


In [92]:
features_dt30.dtypes

Potencia Activa U3    float64
Potencia activa U1    float64
Potencia activa U2    float64
Presión alta 1 U1     float64
Presión alta 1 U2     float64
Presión alta 1 U3     float64
Presión alta 2 U1     float64
Presión alta 2 U2     float64
Presión alta 2 U3     float64
Presión baja 1 U1     float64
Presión baja 1 U2     float64
Presión baja 1 U3     float64
Presión baja 2 U1     float64
Presión baja 2 U2     float64
Presión baja 2 U3     float64
Tª exterior           float64
Tª impulsion U3       float64
Tª impulsión U1       float64
Tª impulsión U2       float64
Tª retorno U1         float64
Tª retorno U2         float64
Tª retorno U3         float64
P30                   float64
dtype: object

#### Standardize

In [94]:
mean= features_dt30.describe().loc["mean"]

In [96]:
std = features_dt30.describe().loc["std"]

In [97]:
features_dt30_std = (features_dt30 - mean) / std

In [171]:
features_dt30_std

,Potencia Activa U3,Potencia activa U1,Potencia activa U2,Presión alta 1 U1,Presión alta 1 U2,Presión alta 1 U3,Presión alta 2 U1,Presión alta 2 U2,Presión alta 2 U3,Presión baja 1 U1,...,Presión baja 2 U2,Presión baja 2 U3,Tª exterior,Tª impulsion U3,Tª impulsión U1,Tª impulsión U2,Tª retorno U1,Tª retorno U2,Tª retorno U3,P30
3,3.732598,0.593484,-1.260525,-0.133213,-0.220696,5.673359,-0.134136,-0.244439,1.166626,-0.144479,...,-0.009122,-0.256930,-0.554475,-1.244431,-0.194850,0.979644,-0.135588,-0.122999,-0.416678,0.090730
4,0.393767,0.581178,-1.260525,-0.137488,-0.220696,4.422928,-0.136971,-0.245951,1.747814,-0.153417,...,-0.013759,-0.244530,-0.297582,-0.686324,-0.561537,0.531223,-0.491249,-0.482838,-0.767829,-0.867459
5,0.736088,0.599637,-1.260525,-0.134638,-0.218343,4.427950,-0.134136,-0.245951,0.043878,-0.153417,...,0.027977,-0.227171,-0.199610,-0.664859,-0.347636,0.770381,-0.288014,-0.277216,-0.592254,-0.728790
6,1.620818,-0.874064,-1.260525,-0.201621,-0.215990,4.417906,-0.168160,-0.241416,0.559021,-0.144479,...,0.023339,-0.274289,-0.521574,-1.566415,-1.050454,-0.246041,-1.177168,-1.151110,-1.445048,-1.490735
7,1.635282,0.467343,-1.260525,-0.144614,-0.219519,-0.207183,-0.144059,-0.247462,2.183705,-0.157886,...,0.009427,-0.274289,-0.200682,-0.729256,-0.653209,0.441538,-0.542058,-0.585649,-0.843075,-1.072033
11,1.702782,-0.914060,-1.260525,-0.204471,-0.215990,-0.207183,-0.173831,-0.244439,2.302584,-0.153417,...,0.009427,3.807625,0.248696,-1.544950,-1.172683,-0.335725,-1.253382,-1.253921,-1.545377,-1.364561
13,1.714835,-0.969439,-1.260525,-0.208746,-0.217167,-0.207183,-0.180919,-0.245951,2.276166,-0.144479,...,0.000153,3.668750,0.345710,-1.459087,-1.142126,-0.455304,-1.354999,-1.356733,-1.620623,-1.479710
16,1.640103,-0.904830,-1.260525,-0.205896,-0.217167,-0.207183,-0.173831,-0.245951,0.585439,-0.148948,...,-0.009122,3.514997,-0.586688,-1.587881,-1.172683,-0.335725,-1.253382,-1.253921,-1.520294,-1.448596
17,1.632871,-0.904830,-1.260525,5.815425,-0.239517,-0.207183,6.534597,-0.271649,0.572230,6.965700,...,-0.027671,-0.276769,-0.361738,-1.630812,-1.233798,-0.365620,-1.304191,-1.253921,-1.570459,-1.391021
21,1.618407,0.476572,-1.260525,-0.144614,-0.223048,-0.207183,-0.145477,-0.255021,0.572230,-0.157886,...,0.009427,-0.264370,-0.715279,-0.900981,-0.469865,0.650802,-0.364227,-0.405730,-0.667500,-0.966803


#### Pipeline

In [191]:
# estimators = [('reduce_dim', PCA()), ('regressor', "passthrough")]

In [288]:
estimators = [('reduce_dim', PCA()), ('regressor', KNeighborsRegressor(weights = "distance"))]

In [289]:
pipe = Pipeline(estimators)

#### GridSearchCV

Models:

In [290]:
model_knn = KNeighborsRegressor(weights = "distance")

iterators:

In [291]:
regressors = [model_knn]

In [292]:
n_components = [1, 2, 3, 4, 5, 6, 7]

In [293]:
knn_power = [1, 2, 3]

In [294]:
knn_neighbors = [9, 11, 13, 15, 17]

In [198]:
# param_grid = [
#     {
#         "regressor": regressors, 
#         "regressor__p": knn_power, 
#         "regressor__n_neighbors": knn_neighbors, 
#         "reduce_dim__n_components": n_components
#     }
        
# ]

In [295]:
param_grid = {
    "regressor__p": knn_power, 
    "regressor__n_neighbors": knn_neighbors, 
    "reduce_dim__n_components": n_components
            }

In [199]:
kf_10 = KFold(n_splits=10, shuffle=True, random_state= 42)

In [296]:
grid = GridSearchCV(pipe, param_grid, scoring= "neg_mean_squared_error", n_jobs= -1, cv = kf_10, error_score= 0, verbose= 3)

In [297]:
grid.fit(features_dt30_std, dt30)

Fitting 10 folds for each of 105 candidates, totalling 1050 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 1050 out of 1050 | elapsed:    6.4s finished
C:\Users\edidd\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning:

The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.



GridSearchCV(cv=KFold(n_splits=10, random_state=51, shuffle=True),
             error_score=0,
             estimator=Pipeline(memory=None,
                                steps=[('reduce_dim',
                                        PCA(copy=True, iterated_power='auto',
                                            n_components=None,
                                            random_state=None,
                                            svd_solver='auto', tol=0.0,
                                            whiten=False)),
                                       ('regressor',
                                        KNeighborsRegressor(algorithm='auto',
                                                            leaf_size=30,
                                                            metric='minkowski',
                                                            metric_params=None,
                                                            n_jobs=None,
                                

In [298]:
cv_results = pd.DataFrame(grid.cv_results_)

In [299]:
cv_results.sort_values(["rank_test_score"])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_reduce_dim__n_components,param_regressor__n_neighbors,param_regressor__p,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
49,0.003791,0.000400,0.002093,0.000300,4,11,2,"{'reduce_dim__n_components': 4, 'regressor__n_neighbors': 11, 'regressor__p': 2}",-0.244369,-0.211899,...,-0.167107,-0.138545,-0.326777,-0.105309,-0.491037,-0.107131,-0.258149,-0.233415,0.111383,1
51,0.004289,0.000459,0.002192,0.000398,4,13,1,"{'reduce_dim__n_components': 4, 'regressor__n_neighbors': 13, 'regressor__p': 1}",-0.256317,-0.203748,...,-0.148703,-0.139793,-0.330313,-0.090671,-0.520934,-0.116439,-0.272866,-0.234736,0.120653,2
50,0.003989,0.000446,0.003091,0.000299,4,11,3,"{'reduce_dim__n_components': 4, 'regressor__n_neighbors': 11, 'regressor__p': 3}",-0.230020,-0.208026,...,-0.169136,-0.133082,-0.343319,-0.096140,-0.502736,-0.112633,-0.265872,-0.235773,0.117222,3
48,0.004589,0.000660,0.002192,0.000400,4,11,1,"{'reduce_dim__n_components': 4, 'regressor__n_neighbors': 11, 'regressor__p': 1}",-0.261946,-0.204175,...,-0.166227,-0.139857,-0.318759,-0.096390,-0.507643,-0.121424,-0.273019,-0.235785,0.114426,4
47,0.004389,0.000798,0.004886,0.003744,4,9,3,"{'reduce_dim__n_components': 4, 'regressor__n_neighbors': 9, 'regressor__p': 3}",-0.232023,-0.185755,...,-0.180921,-0.133185,-0.350424,-0.082438,-0.494461,-0.116270,-0.292575,-0.236457,0.118448,5
46,0.005884,0.005035,0.002395,0.000488,4,9,2,"{'reduce_dim__n_components': 4, 'regressor__n_neighbors': 9, 'regressor__p': 2}",-0.244408,-0.185329,...,-0.175942,-0.130307,-0.348204,-0.089475,-0.509901,-0.106019,-0.289918,-0.236644,0.121775,6
52,0.004188,0.000747,0.002294,0.000457,4,13,2,"{'reduce_dim__n_components': 4, 'regressor__n_neighbors': 13, 'regressor__p': 2}",-0.245983,-0.225270,...,-0.152755,-0.138979,-0.356877,-0.093174,-0.491198,-0.119575,-0.248200,-0.236897,0.115565,7
55,0.005885,0.003552,0.002492,0.000499,4,15,2,"{'reduce_dim__n_components': 4, 'regressor__n_neighbors': 15, 'regressor__p': 2}",-0.225770,-0.230883,...,-0.140481,-0.145750,-0.350063,-0.087925,-0.502591,-0.119519,-0.260936,-0.236914,0.119121,8
53,0.004190,0.000400,0.003689,0.000897,4,13,3,"{'reduce_dim__n_components': 4, 'regressor__n_neighbors': 13, 'regressor__p': 3}",-0.216602,-0.226272,...,-0.165954,-0.136760,-0.339838,-0.093079,-0.502771,-0.120883,-0.257631,-0.237172,0.116911,9
54,0.003988,0.000003,0.002295,0.000456,4,15,1,"{'reduce_dim__n_components': 4, 'regressor__n_neighbors': 15, 'regressor__p': 1}",-0.269598,-0.202930,...,-0.145324,-0.145701,-0.347149,-0.087486,-0.515679,-0.124425,-0.265104,-0.238258,0.120714,10


In [300]:
grid.best_estimator_

Pipeline(memory=None,
         steps=[('reduce_dim',
                 PCA(copy=True, iterated_power='auto', n_components=4,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('regressor',
                 KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                     metric='minkowski', metric_params=None,
                                     n_jobs=None, n_neighbors=11, p=2,
                                     weights='distance'))],
         verbose=False)

In [301]:
grid.best_score_

-0.23341498045716527

#### 10-fold outer 10-fold inner nested CV

In [233]:
kf_10 = KFold(n_splits=10, shuffle=True, random_state= 42)

In [234]:
grid = GridSearchCV(pipe, param_grid, scoring= "neg_mean_squared_error", n_jobs= -1, cv = kf_10, error_score= 0, verbose= 3)

In [235]:
kf_10_nested = KFold(n_splits=10, shuffle=True, random_state= 13)

In [236]:
nested_cv = cross_val_score(estimator= grid, 
                            X= features_dt30_std, 
                            y= dt30, 
                            scoring="neg_mean_squared_error", 
                            cv= kf_10_nested, 
                            n_jobs= -1, 
                            verbose= 3
                           )

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   21.6s remaining:    9.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   29.1s finished


In [237]:
nested_cv

array([-0.33592284, -0.18399834, -0.39296864, -0.360954  , -0.11689732,
       -0.41802628, -0.20882262, -0.13526649, -0.18921267, -0.24927942])

In [238]:
nested_cv.mean()

-0.2591348605524314

In [240]:
nested_cv.std()

0.10399973432162425

In [241]:
nested_cv_se = nested_cv.std() / (10 ** 0.5)

KNN: 95% confidence interval

In [270]:
print("MSE: %0.2f" %abs(nested_cv.mean()) + " +/-" + "%0.2f" % (2*nested_cv_se))
# "MAE: %0.2f (+/- %0.2f)" % (abs(RF_scores.mean()), RF_scores.std() * 2)

MSE: 0.26 +/-0.07


#### Validating the estimated test error

##### Splitting train and test sets

In [254]:
X_train, X_test, y_train, y_test = train_test_split(features_dt30_std, dt30, test_size=0.3, random_state=42)

In [255]:
X_train

,Potencia Activa U3,Potencia activa U1,Potencia activa U2,Presión alta 1 U1,Presión alta 1 U2,Presión alta 1 U3,Presión alta 2 U1,Presión alta 2 U2,Presión alta 2 U3,Presión baja 1 U1,...,Presión baja 2 U2,Presión baja 2 U3,Tª exterior,Tª impulsion U3,Tª impulsión U1,Tª impulsión U2,Tª retorno U1,Tª retorno U2,Tª retorno U3,P30
101,0.343143,0.522722,-1.260525,-0.140339,-0.227754,-0.207183,-0.141224,-0.251997,1.523264,-0.140010,...,-0.004484,-0.244530,-0.908765,-0.299943,-0.042063,1.069328,0.042243,-0.045891,-0.316349,-0.685670
5,0.736088,0.599637,-1.260525,-0.134638,-0.218343,4.427950,-0.134136,-0.245951,0.043878,-0.153417,...,0.027977,-0.227171,-0.199610,-0.664859,-0.347636,0.770381,-0.288014,-0.277216,-0.592254,-0.728790
185,-0.695873,0.510415,0.895091,-0.137488,-0.073657,-0.207183,-0.139806,-0.056992,-0.590145,-0.184700,...,-0.227074,-0.115576,0.444117,0.816270,-0.072621,-0.574883,0.245479,0.236840,0.837431,1.135648
155,-0.695873,0.344278,0.858734,-0.157441,-0.086596,-0.207183,-0.158236,-0.072109,-0.722233,-0.175762,...,-0.194613,-0.123015,-0.908130,0.623079,1.363572,0.710591,1.541104,1.547682,0.586609,0.507258
192,-0.695873,0.781158,0.903301,7.608282,6.036100,-0.207183,7.739618,7.764397,-0.590145,6.974638,...,7.466174,6.862861,0.926605,0.945063,0.049609,-0.425410,-0.110183,-0.122999,0.887596,0.892610
222,-0.695873,0.784234,0.950213,-0.113260,-0.063070,-0.207183,-0.112871,-0.041875,-0.537310,-0.157886,...,-0.213162,-0.095737,0.926928,1.181185,0.416296,-0.275936,0.423310,0.442462,1.213664,1.332872
193,-0.695873,0.685783,0.924411,-0.127512,6.086681,-0.207183,-0.127047,-0.053969,-0.563728,-0.113196,...,-0.180701,-0.103176,-0.554475,1.009460,1.546916,0.949749,1.617317,1.650493,0.987924,1.142263
289,-0.695873,-1.086350,0.701578,-0.220148,-0.511244,-0.207183,-0.193678,-0.102342,-0.867530,-0.171293,...,-0.278084,-0.160214,-1.392543,-0.621928,-0.683766,-1.651095,-0.618271,-0.585649,-0.742747,-0.237813
153,-0.695873,-1.003282,0.294615,-0.180243,-0.114828,-0.207183,-0.212108,-0.143157,-0.788277,-0.198107,...,-0.217799,-0.147814,-1.005674,0.494285,-0.622652,-0.993410,-0.440440,-0.431432,0.310705,-0.498737
59,1.654568,-0.895600,-1.260525,-0.200195,-0.219519,-0.207183,-0.170995,-0.245951,0.638274,-0.140010,...,-0.060132,-0.269329,-0.555756,-1.416156,-0.775438,-0.036778,-0.923124,-0.971191,-1.244390,-1.182282


In [256]:
y_train

101   -3.730000
5     -3.570000
185   -2.400000
155   -2.630000
192   -2.398904
222   -2.830000
193   -3.200000
289   -1.470000
153   -1.260000
59    -0.900000
234   -2.730000
214   -2.470000
128   -2.430000
285   -1.440000
11    -1.000000
88    -0.730000
283   -0.960000
270   -1.800000
202   -2.470000
97    -1.230000
182   -1.200000
121   -2.900000
263   -1.230000
276   -1.330000
63    -1.570000
34    -2.830000
67    -1.370000
58    -0.700000
74    -0.730000
163   -3.870000
146   -1.830000
156   -0.970000
47    -3.560000
220   -2.430000
129   -1.205216
217   -4.160000
157   -0.500000
83    -2.400000
150   -0.900000
235   -4.170000
131   -2.760000
190   -2.300000
291   -1.630000
111   -0.660000
73    -0.610556
237   -3.870000
213   -4.500000
113   -3.460000
200   -0.900000
56    -2.960000
293   -1.170000
184   -1.330000
3     -2.530000
249   -4.168007
138   -2.130000
254   -2.700000
223   -4.100000
136   -2.930000
68    -2.970000
93    -3.230000
210   -2.730000
186   -3.130000
33    -3

##### Hyperparameter tunning in X_train, y_train

In [257]:
param_grid = [
    {
        "regressor__p": knn_power, 
        "regressor__n_neighbors": knn_neighbors, 
        "reduce_dim__n_components": n_components
    }
        
]

In [258]:
kf_10 = KFold(n_splits=10, shuffle=True, random_state= 42)

In [259]:
grid = GridSearchCV(pipe, param_grid, scoring= "neg_mean_squared_error", n_jobs= -1, cv = kf_10, error_score= 0, verbose= 3)

In [260]:
grid.fit(X_train, y_train)

Fitting 10 folds for each of 105 candidates, totalling 1050 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 1050 out of 1050 | elapsed:    6.9s finished
C:\Users\edidd\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning:

The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.



GridSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
             error_score=0,
             estimator=Pipeline(memory=None,
                                steps=[('reduce_dim',
                                        PCA(copy=True, iterated_power='auto',
                                            n_components=None,
                                            random_state=None,
                                            svd_solver='auto', tol=0.0,
                                            whiten=False)),
                                       ('regressor',
                                        KNeighborsRegressor(algorithm='auto',
                                                            leaf_size=30,
                                                            metric='minkowski',
                                                            metric_params=None,
                                                            n_jobs=None,
                                

In [261]:
cv_results = pd.DataFrame(grid.cv_results_)

In [262]:
cv_results.sort_values(["rank_test_score"])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_reduce_dim__n_components,param_regressor__n_neighbors,param_regressor__p,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
30,0.005086,0.002337,0.002394,0.000489,3,9,1,"{'reduce_dim__n_components': 3, 'regressor__n_neighbors': 9, 'regressor__p': 1}",-0.094481,-0.117848,...,-0.326605,-0.467305,-0.169210,-0.081289,-0.302684,-0.199120,-0.183737,-0.207673,0.115360,1
31,0.006384,0.005630,0.002292,0.000638,3,9,2,"{'reduce_dim__n_components': 3, 'regressor__n_neighbors': 9, 'regressor__p': 2}",-0.099479,-0.114444,...,-0.331033,-0.478161,-0.190898,-0.091407,-0.294783,-0.192011,-0.170183,-0.210120,0.116056,2
33,0.003889,0.000829,0.002095,0.000300,3,11,1,"{'reduce_dim__n_components': 3, 'regressor__n_neighbors': 11, 'regressor__p': 1}",-0.095369,-0.117125,...,-0.327591,-0.470182,-0.170945,-0.078227,-0.337514,-0.191938,-0.193749,-0.211162,0.119767,3
47,0.005483,0.004847,0.005086,0.007628,4,9,3,"{'reduce_dim__n_components': 4, 'regressor__n_neighbors': 9, 'regressor__p': 3}",-0.082853,-0.184857,...,-0.314525,-0.454647,-0.174537,-0.064853,-0.307362,-0.223637,-0.144891,-0.211506,0.111718,4
49,0.004286,0.000895,0.002694,0.000637,4,11,2,"{'reduce_dim__n_components': 4, 'regressor__n_neighbors': 11, 'regressor__p': 2}",-0.100995,-0.185687,...,-0.330460,-0.468368,-0.165853,-0.066372,-0.279549,-0.213709,-0.149032,-0.212326,0.112096,5
34,0.004786,0.003085,0.002094,0.000300,3,11,2,"{'reduce_dim__n_components': 3, 'regressor__n_neighbors': 11, 'regressor__p': 2}",-0.099754,-0.118798,...,-0.358468,-0.474144,-0.186481,-0.085336,-0.305875,-0.192251,-0.174011,-0.212733,0.119604,6
36,0.003990,0.000632,0.002094,0.000300,3,13,1,"{'reduce_dim__n_components': 3, 'regressor__n_neighbors': 13, 'regressor__p': 1}",-0.101458,-0.123556,...,-0.343588,-0.481875,-0.184235,-0.073170,-0.302420,-0.185410,-0.191096,-0.213145,0.119260,7
50,0.003988,0.000446,0.004387,0.005539,4,11,3,"{'reduce_dim__n_components': 4, 'regressor__n_neighbors': 11, 'regressor__p': 3}",-0.104780,-0.186259,...,-0.338335,-0.464838,-0.166685,-0.070183,-0.267039,-0.221458,-0.150885,-0.213363,0.110486,8
37,0.004787,0.003449,0.002594,0.001277,3,13,2,"{'reduce_dim__n_components': 3, 'regressor__n_neighbors': 13, 'regressor__p': 2}",-0.106575,-0.120051,...,-0.376725,-0.474374,-0.192553,-0.085441,-0.287368,-0.180978,-0.183163,-0.213805,0.119905,9
38,0.004589,0.001352,0.002592,0.000662,3,13,3,"{'reduce_dim__n_components': 3, 'regressor__n_neighbors': 13, 'regressor__p': 3}",-0.106529,-0.116760,...,-0.386500,-0.462288,-0.204003,-0.084747,-0.288517,-0.176192,-0.182334,-0.213932,0.119104,10


In [263]:
grid.best_estimator_

Pipeline(memory=None,
         steps=[('reduce_dim',
                 PCA(copy=True, iterated_power='auto', n_components=3,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('regressor',
                 KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                     metric='minkowski', metric_params=None,
                                     n_jobs=None, n_neighbors=9, p=1,
                                     weights='distance'))],
         verbose=False)

In [264]:
grid.best_score_

-0.20767302921711184

##### Predicting with best model in X_test and analysing test errors

In [265]:
prediction_knn= grid.predict(X_test)

MSE:

In [271]:
mean_squared_error(y_test, prediction_knn)

0.2991370281344988

##### Plotting errors

In [277]:
error = abs(y_test - prediction_knn)

In [278]:
error.mean()

0.3959377804645889

In [282]:
results= pd.DataFrame(
    {"y": y_test, 
     "y_hat": prediction_knn, 
     "error": error, 
     
    }
)

In [283]:
results = results.sort_values("y")

In [284]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= results.y, 
                         y= results.error, 
                         mode= "markers",
                         name= "error"))
fig.update_layout(
    title= {
        'text': "Prediction error Knn",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [285]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= results.y, 
                         y= results.y_hat, 
                         mode= "markers",
                         name= "prediction knn"))

fig.update_layout(
    title= {
        'text': "Prediction error knnn",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [286]:
fig = go.Figure()
fig.add_trace(go.Scatter(
                         y= results.y, 
                         mode= "markers",
                         name= "real y"))
fig.add_trace(go.Scatter(
                         y= results.y_hat, 
                         mode= "markers",
                         name= "prediction"))

fig.update_layout(
    title= {
        'text': "Prediction error knn",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

##### Looping over different seeds

In [267]:
seeds= [223, 43, 6, 456, 51]

In [268]:
mse = []
for i in seeds:
    X_train, X_test, y_train, y_test = train_test_split(features_dt30_std, dt30, test_size=0.3, random_state=i)
    kf_10 = KFold(n_splits=10, shuffle=True, random_state= i)
    grid = GridSearchCV(pipe, param_grid, scoring= "neg_mean_squared_error", n_jobs= -1, cv = kf_10, error_score= 0, verbose= 3)
    grid.fit(X_train, y_train)
    prediction_knn= grid.predict(X_test)
    mse.append(mean_squared_error(y_test, prediction_knn))

Fitting 10 folds for each of 105 candidates, totalling 1050 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 1050 out of 1050 | elapsed:    6.3s finished
C:\Users\edidd\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning:

The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 10 folds for each of 105 candidates, totalling 1050 fits


[Parallel(n_jobs=-1)]: Done 504 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 1050 out of 1050 | elapsed:    2.6s finished
C:\Users\edidd\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning:

The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 10 folds for each of 105 candidates, totalling 1050 fits


[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 1050 out of 1050 | elapsed:    2.7s finished
C:\Users\edidd\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning:

The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 10 folds for each of 105 candidates, totalling 1050 fits


[Parallel(n_jobs=-1)]: Done 360 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 1050 out of 1050 | elapsed:    2.8s finished
C:\Users\edidd\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning:

The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 10 folds for each of 105 candidates, totalling 1050 fits


[Parallel(n_jobs=-1)]: Done 360 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 1050 out of 1050 | elapsed:    2.6s finished
C:\Users\edidd\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning:

The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.



In [269]:
mse

[0.1866606963557419,
 0.19299223625560866,
 0.27179065560699484,
 0.25643423216898853,
 0.2991370281344988]

#### Hyperparameter tunning and model assessment SVR

In [307]:
def get_ncv_metric(model, X, y, n_jobs, **kwargs):
    estimators = [('reduce_dim', PCA()), ('regressor', model)]
    pipe = Pipeline(estimators)
    param_grid = {"reduce_dim__n_components": n_components}
    for key, value in kwargs.items(): 
        param_grid["regressor__" + str(key)] = value
    kf_10 = KFold(n_splits=10, shuffle=True, random_state= 42)
    grid = GridSearchCV(
        pipe, 
        param_grid, 
        scoring= "neg_mean_squared_error", 
        n_jobs= n_jobs, 
        cv = kf_10, 
        error_score= 0, 
        verbose= 3
    )
    kf_10_nested = KFold(n_splits=10, shuffle=True, random_state= 13)
    nested_cv = cross_val_score(
        estimator= grid, 
        X= X, 
        y= y, 
        scoring="neg_mean_squared_error", 
        cv= kf_10_nested, 
        n_jobs= n_jobs, 
        verbose= 3
    )
    return nested_cv

In [308]:
svr= SVR(kernel= "rbf") #, C= 1, gamma= "auto")

In [311]:
c_list= [0.1, 1, 10, 100]

In [312]:
g_list= [0.1, 1, 10, "auto", "scale"]

In [313]:
ncv_svr= get_ncv_metric(svr, features_dt30_std, dt30, -1, C= c_list, gamma= g_list)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   57.8s remaining:   24.7s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.2min finished


In [314]:
ncv_svr.mean()

-0.24327906225339077

In [315]:
ncv_svr.std()

0.08812295938727521

In [316]:
ncv_svr_se = ncv_svr.std() / (10 ** 0.5)

SVR rbf: 95% confidence interval

In [318]:
print("MSE: %0.2f" %abs(ncv_svr.mean()) + " +/- " + "%0.2f" % (2*ncv_svr_se))
# "MAE: %0.2f (+/- %0.2f)" % (abs(RF_scores.mean()), RF_scores.std() * 2)

MSE: 0.24 +/- 0.06


#### Hyperparameter tunning and model assessment RF

In [325]:
rf = RandomForestRegressor(criterion='mse', max_depth=None, min_samples_split=2, n_jobs=3)

In [323]:
m_list = ["auto", "sqrt"]

In [324]:
n_trees = [50, 100, 200, 300]

In [326]:
ncv_rf= get_ncv_metric(rf, features_dt30_std, dt30, 3, max_features= m_list, n_estimators= n_trees)

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:  8.1min finished


In [327]:
ncv_rf.mean()

-0.23444018314109907

In [328]:
ncv_rf.std()

0.0747288160276617

In [329]:
ncv_rf_se = ncv_rf.std() / (10 ** 0.5)

RF: 95% confidence interval

In [330]:
print("MSE: %0.2f" %abs(ncv_rf.mean()) + " +/- " + "%0.2f" % (2*ncv_rf_se))
# "MAE: %0.2f (+/- %0.2f)" % (abs(RF_scores.mean()), RF_scores.std() * 2)

MSE: 0.23 +/- 0.05


##### RF with original features (no PCA)

In [332]:
param_grid= {
    "max_features": ["auto", "sqrt"],
    "n_estimators": [300, 400, 500]
}

In [333]:
kf_10 = KFold(n_splits=10, shuffle=True, random_state= 42)
    

In [334]:
grid = GridSearchCV(
    rf, 
    param_grid, 
    scoring= "neg_mean_squared_error", 
    n_jobs= 2, 
    cv = kf_10, 
    error_score= 0, 
    verbose= 5
)

kf_10_nested = KFold(n_splits=10, shuffle=True, random_state= 13)

nested_cv = cross_val_score(
    estimator= grid, 
    X= features_dt30_std, 
    y= dt30, 
    scoring="neg_mean_squared_error", 
    cv= kf_10_nested, 
    n_jobs= 2, 
    verbose= 5
)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:  3.6min remaining:    0.0s
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:  3.6min finished


In [335]:
ncv_rf_f0 = nested_cv

In [336]:
ncv_rf_f0.mean()

-0.2213923076395173

In [337]:
ncv_rf_f0.std()

0.08399767471640107

In [339]:
ncv_rf_f0_se = ncv_rf_f0.std() / (10 ** 0.5)

RF without PCA: 95% confidence interval

In [340]:
print("MSE: %0.2f" %abs(ncv_rf_f0.mean()) + " +/- " + "%0.2f" % (2*ncv_rf_f0_se))
# "MAE: %0.2f (+/- %0.2f)" % (abs(RF_scores.mean()), RF_scores.std() * 2)

MSE: 0.22 +/- 0.05


### Splitting train and test sets

In [48]:
X_train, X_test, y_train, y_test = train_test_split(features_dt30, dt30, test_size=0.3, random_state=42)

In [49]:
X_train

,Potencia Activa U3,Potencia activa U1,Potencia activa U2,Presión alta 1 U1,Presión alta 1 U2,Presión alta 1 U3,Presión alta 2 U1,Presión alta 2 U2,Presión alta 2 U3,Presión baja 1 U1,...,Presión baja 2 U2,Presión baja 2 U3,Tª exterior,Tª impulsion U3,Tª impulsión U1,Tª impulsión U2,Tª retorno U1,Tª retorno U2,Tª retorno U3,P30
101,22.00,88.35,0.40,26.7,14.1,-88.8,27.2,14.2,31.1,8.6,...,13.5,9.4,23.715500,17.4,13.5,18.1,18.7,18.1,18.1,118.150000
5,30.15,89.60,0.40,27.1,14.9,3.5,27.7,14.6,19.9,8.3,...,14.2,10.1,26.429167,15.7,12.5,17.1,17.4,17.2,17.0,116.683333
185,0.45,88.15,92.30,26.9,27.2,-88.8,27.3,27.1,15.1,7.6,...,8.7,14.6,28.892467,22.6,13.4,12.6,19.5,19.2,22.7,180.100000
155,0.45,85.45,90.75,25.5,26.1,-88.8,26.0,26.1,14.1,7.8,...,9.4,14.3,23.717933,21.7,18.1,16.9,24.6,24.3,21.7,158.726052
192,0.45,92.55,92.65,570.4,546.6,-88.8,583.1,544.5,15.1,167.8,...,174.6,296.0,30.738767,23.2,13.8,13.1,18.1,17.8,22.9,171.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,49.55,67.00,0.40,25.9,14.1,-88.8,24.0,14.0,24.8,8.1,...,13.5,8.5,26.176300,13.4,11.8,15.7,15.8,15.7,15.4,112.758333
226,0.45,93.00,94.20,28.9,28.0,-88.8,29.4,28.0,15.6,8.1,...,8.8,15.4,33.330400,24.3,14.4,13.1,19.5,19.2,24.2,187.691667
104,46.45,88.40,0.40,26.9,14.5,-88.8,27.3,14.6,34.1,8.6,...,13.2,9.3,25.933433,16.2,13.2,17.8,18.3,17.9,17.7,126.733333
144,0.45,92.70,93.75,28.6,27.7,-88.8,29.1,27.6,15.3,8.2,...,8.9,14.6,31.357933,21.8,13.0,12.8,18.0,17.6,21.7,162.516667


In [50]:
y_train

101   -3.730000
5     -3.570000
185   -2.400000
155   -2.630000
192   -2.398904
         ...   
45    -3.160000
226   -2.540000
104   -3.130000
144   -2.340000
140   -1.270000
Name: DT30, Length: 182, dtype: float64

### PCA

This is to standardize the features, get the eigenvetors and project the features to the pc space

In [38]:
def pc_transform(features):
    # Standardize
    mean = features.describe().loc["mean"]
    std = features.describe().loc["std"]
    features_std = (features - mean) / std
    # PCA
    columns_names = []
    for i in range(0, len(features.columns)):
        columns_names.append("PC" + str(i + 1))
    pc = PCA().fit(features_std)
    pc_features = pd.DataFrame(pc.transform(features_std), columns= columns_names)
    output = {"pc_components": pc.components_, "pc_features": pc_features}
    return output

In [51]:
X_train_pc = pc_transform(X_train)["pc_features"]

In [52]:
X_train_pc.iloc[:,:3]

,PC1,PC2,PC3
0,-0.766287,-0.989856,2.043104
1,-0.722468,-1.568929,1.192962
2,-0.236006,1.896191,-1.093111
3,-0.391066,2.818699,0.283050
4,20.021570,0.744261,1.300436
...,...,...,...
177,-0.738765,-2.664760,1.089228
178,-0.168313,2.594255,-0.871127
179,-0.831072,-1.393564,2.597212
180,-0.195083,1.441078,-1.090379


Fitting the models with the selected hyperparameters

Models:

In [44]:
model_svr_c1 = SVR(kernel= "rbf", C= 1, gamma= "auto") # with 3 PCs

In [45]:
model_svr_c10 = SVR(kernel= "rbf", C= 10, gamma= "auto") # with 5 PCs

In [53]:
model_svr_c1.fit(X_train_pc.iloc[:,:3], y_train)

SVR(C=1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [54]:
model_svr_c10.fit(X_train_pc.iloc[:,:5], y_train)

SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

Applying pc transformation to X_test

In [55]:
mean = X_train.describe().loc["mean"]
std = X_train.describe().loc["std"]
X_test_std = (X_test - mean) / std

In [62]:
X_test_pc = pd.DataFrame(pc_transform(X_train)["pc_components"].dot(X_test_std.T).T)

Predicting on X_test_pc

In [64]:
prediction_c1 = model_svr_c1.predict(X_test_pc.iloc[:,:3])

In [65]:
prediction_c10 = model_svr_c10.predict(X_test_pc.iloc[:,:5])

MSE:

In [66]:
mean_squared_error(y_test, prediction_c1)

0.2590717410819972

In [67]:
mean_squared_error(y_test, prediction_c10)

0.27885978297624836

Plotting errors

In [68]:
error_c1 = abs(y_test - prediction_c1)

In [83]:
error_c1.mean()

0.3302423421285744

In [69]:
error_c10 = abs(y_test - prediction_c10)

In [84]:
error_c10.mean()

0.33798840822143356

In [71]:
results= pd.DataFrame(
    {"y": y_test, 
     "y_hat_c1": prediction_c1, 
     "error_c1": error_c1, 
     "y_hat_c10": prediction_c10, 
     "error_c10": error_c10
    }
)

In [78]:
results = results.sort_values("y")

In [79]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= results.y, 
                         y= results.error_c1, 
                         mode= "markers",
                         name= "error with C=1"))
fig.add_trace(go.Scatter(x= results.y, 
                         y= results.error_c10, 
                         mode= "markers",
                         name= "error with C=10"))
fig.update_layout(
    title= {
        'text': "Prediction error of DT 30min",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [74]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= results.y, 
                         y= results.y_hat_c1, 
                         mode= "markers",
                         name= "prediction with c=1"))
fig.add_trace(go.Scatter(x= results.y, 
                         y= results.y_hat_c10, 
                         mode= "markers",
                         name= "prediction with C=10"))
fig.update_layout(
    title= {
        'text': "Prediction error of DT 30min",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [82]:
fig = go.Figure()
fig.add_trace(go.Scatter(
                         y= results.y, 
                         mode= "markers",
                         name= "real y"))
fig.add_trace(go.Scatter(
                         y= results.y_hat_c1, 
                         mode= "markers",
                         name= "prediction with c=1"))
fig.add_trace(go.Scatter(
                         y= results.y_hat_c10, 
                         mode= "markers",
                         name= "prediction with C=10"))
fig.update_layout(
    title= {
        'text': "Prediction error of DT 30min",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()